
## Building a Conversational Chatbot in your local with Llama 3.2 using Ollama

This notebook demonstrates a streamlined workflow for building a **Retrieval-Augmented Generation (RAG)** system locally using **Llama 3.2** via **Ollama**. The goal is to enable efficient document-based question answering by integrating document ingestion, vector database storage, and a conversational retrieval system. 

The following steps are covered:

1. **Document Preparation and Splitting**: PDF documents from a specified directory are loaded, and text passages are split into manageable chunks using `RecursiveCharacterTextSplitter`. This ensures that documents are split based on size and overlap to optimize retrieval accuracy.

2. **Ingesting Documents into Vector Database**: The split documents are embedded using `HuggingFaceEmbeddings` and stored in a local FAISS vector database, facilitating fast and scalable document retrieval.

3. **Building the Conversation Chain**: A conversational chain is built using **Llama 3.2** to retrieve relevant information based on user queries. This chain ensures that the responses are accurate, concise, and relevant to the context of the chat history, while managing session history to maintain the flow of conversation.

4. **Similarity Score Calculation**: To evaluate the relevancy of the system’s responses, the notebook includes a function to calculate the similarity score between the generated answers and the source documents using `SentenceTransformer`.

The notebook is designed for local execution, leveraging the performance and capabilities of Llama 3.2 via the **Ollama** API to offer a robust and efficient Q&A solution.

Below cell imports the required libraries to run this notebook.

In [2]:
install_packages=False


In [4]:
if install_packages==True:
    !pip install -r requirements.txt


In [1]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from sentence_transformers import SentenceTransformer, util
#from htmlTemplate import css, bot_template, user_template
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains import create_history_aware_retriever
from langchain_huggingface import HuggingFaceEmbeddings

/Users/vikrambhat/Documents/GitHub/RAG-Implementation-with-ConversationUI/RAG Implementation Notebook/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


### Enter your pdf file name below


### Step 1: Prepare documents and their metadata
The `prepare_and_split_docs` function loads PDF documents from a specified directory using `DirectoryLoader`. It then splits the loaded documents into smaller text chunks using `RecursiveCharacterTextSplitter` with a chunk size of 512 and an overlap of 256. The splitter ensures that document metadata is preserved and splits content based on specified separators like new lines and spaces. The function finally returns the split documents and prints the total number of text passages created. This process aids in efficient document handling for downstream tasks.

In [7]:
def prepare_and_split_docs(pdf_directory):
    # Load the documents
    loader = DirectoryLoader(pdf_directory, glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader)
    documents = loader.load()

    # Initialize a text splitter
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,  # Use the smaller chunk size here to avoid repeating splitting logic
        chunk_overlap=256,
        disallowed_special=(),
        separators=["\n\n", "\n", " "]
    )

    # Split the documents and keep metadata
    split_docs = splitter.split_documents(documents)

    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs


### Step 3: Ingest into Vector Database locally

The `ingest_into_vectordb` function is designed for processing and indexing a collection of documents into a vector database using FAISS (Facebook AI Similarity Search) for efficient similarity searches. It operates as follows:

1. **Embedding Creation**: It generates embeddings for the input documents (`split_docs`) using the Hugging Face model `'sentence-transformers/all-MiniLM-L6-v2'`. This model is specifically chosen for its efficiency in creating sentence-level embeddings and is set to run on the CPU.

2. **Vector Database Indexing**: Utilizes the generated embeddings to create a FAISS vector database. FAISS is used for its ability to efficiently handle large-scale similarity searches and clustering of dense vectors.

3. **Local Storage**: After creating the vector database, the function saves it locally to the path specified by `DB_FAISS_PATH`, ensuring the data can be easily accessed for future similarity searches or retrieval tasks.

The primary purpose of this function is to transform textual data into a structured, searchable vector format, facilitating efficient and scalable retrieval tasks such as document similarity searches or clustering.

In [6]:
def ingest_into_vectordb(split_docs):
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    db = FAISS.from_documents(split_docs, embeddings)

    DB_FAISS_PATH = 'vectorstore/db_faiss'
    db.save_local(DB_FAISS_PATH)
    print("Documents are inserted into FAISS vectorstore")
    return db

### Step 4: Set up Conversation Chain using LLM
The `get_conversation_chain(retriever)` function creates a stateful conversational RAG system.

1. It initializes the `llama3.2` model and defines two prompts:
   - A contextualization prompt to handle the user's query in light of the chat history.
   - A system prompt for answering concisely with 2-3 sentences based on retrieved documents.

2. It builds a `history_aware_retriever` using the retriever, LLM, and the contextualization prompt to ensure responses are context-aware.

3. A `question_answer_chain` is set up to respond with answers limited to 50 words.

4. These components are combined into a RAG chain using `create_retrieval_chain`.

5. To manage chat history across sessions, it defines `get_session_history`, which stores and retrieves message history by session ID.

6. Finally, a `RunnableWithMessageHistory` integrates the RAG chain with chat history management, ensuring the bot maintains state and provides contextually relevant responses throughout the conversation.

This function sets up a sophisticated conversational AI system combining the LLaMA model for language generation and a vector database for information retrieval, enhanced with a callback manager for additional processing and a conversation memory buffer for context management.

In [13]:
def get_conversation_chain(retriever):
    llm = Ollama(model="llama3.2")
    contextualize_q_system_prompt = (
        "Given the chat history and the latest user question, "
        "provide a response that directly addresses the user's query based on the provided  documents. "
        "Do not rephrase the question or ask follow-up questions."
    )


    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )


    ### Answer question ###
    system_prompt = (
        "As a personal chat assistant, provide accurate and relevant information based on the provided document in 2-3 sentences. "
        "Answe should be limited to 50 words and 2-3 sentences.  do not prompt to select answers or do not formualate a stand alone question. do not ask questions in the response. "
        "{context}"
    )

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


    ### Statefully manage chat history ###
    store = {}


    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]


    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    print("Conversational chain created")
    return conversational_rag_chain

### Step 5: Calculate Document Similarity in the LLMs Response
The `calculate_similarity_score` function computes the cosine similarity between a given answer and a list of context documents using Sentence Transformers. It first encodes the answer and context documents into embeddings. Then, it calculates the cosine similarities between the answer embedding and the context embeddings. The function returns the maximum similarity score, indicating how closely the answer relates to the most relevant context document. Scores range from 0 (no similarity) to 1 (perfect similarity), with higher scores reflecting better alignment with the context.

Essentially, this function serves as a mechanism to check the alignment of the chatbot's response with the information in the source documents, ensuring the response's accuracy and relevance.

In [9]:
def calculate_similarity_score(answer: str, context_docs: list) -> float:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    context_docs = [doc.page_content for doc in context_docs]
    # Encode the answer and context documents
    answer_embedding = model.encode(answer, convert_to_tensor=True)
    context_embeddings = model.encode(context_docs, convert_to_tensor=True)

    # Calculate cosine similarities
    similarities = util.pytorch_cos_sim(answer_embedding, context_embeddings)

    # Return the maximum similarity score from the context documents
    max_score = similarities.max().item() 
    return max_score

Now that we have crafted all the necessary functions, it's time to put them into action and test their functionality.

In [14]:
pdf_directory="data_directory"
split_docs=prepare_and_split_docs(pdf_directory)
vector_db= ingest_into_vectordb(split_docs)


retriever =vector_db.as_retriever()
conversational_rag_chain=get_conversation_chain(retriever)


100%|██████████| 1/1 [00:00<00:00, 20.03it/s]


Documents are split into 3 passages


/Users/vikrambhat/Documents/GitHub/RAG-Implementation-with-ConversationUI/RAG Implementation Notebook/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Documents are inserted into FAISS vectorstore
Conversational chain created


### Ask your Question

We created a conversational chain and now ready to chat with your own data. 


### Question 1

In [16]:
user_question="Can you summarise the documents ?"

In [17]:

qa1=conversational_rag_chain.invoke(
    {"input": user_question},
    config={
        "configurable": {"session_id": "abc123"}
    }
)
print(qa1["answer"])

Vikram Bhat has 8 years of experience in data science technologies with expertise in retrieving augmented generation (RAG) systems, machine learning, and data visualization. He holds an MSc in Data Science and Analytics from University College Cork, Ireland, and has a strong background in programming languages like Python, R, and SQL.


We have now received an answer for a provided question. We can also view the conversation history and source documents in the response.


In [18]:
print("Conversation Chain")
print(qa1)

Conversation Chain
{'input': 'Can you summarise the documents ?', 'chat_history': [], 'context': [Document(metadata={'source': 'data_directory/Vikram Bhat - Experienced Data Scientist.pdf', 'page': 1}, page_content='WatsonX LLMS platform, focusing on enhancing language processing and information retrieval capabilities. ● Employed Elasticsearch for the vector storage solution, significantly enhancing data indexing and search response efficiency, which resulted in faster and more accurate query handling. ● Developed models for Regression, Prediction, Clustering, and Time-Series analysis, catering to sectors like Wealth Management, Utilities, and Healthcare by delving into specific industry challenges. ● Managed comprehensive datasets, ensuring meticulous preparation, and cleansing to construct and operationalize machine learning models within Watson Studio, facilitating real-time scoring in Python via Jupyter notebooks. ● Created interactive R-shiny dashboards for dynamic visualization a

### Calculate Similarity score between the LLM Response and context documents

In [19]:

answer = qa1["answer"]
context_docs = qa1["context"]
similarity_score = calculate_similarity_score(answer, context_docs)

print("Context Similarity Score:", round(similarity_score,2))

/Users/vikrambhat/Documents/GitHub/RAG-Implementation-with-ConversationUI/RAG Implementation Notebook/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Context Similarity Score: 0.53


### Sumamry
This notebook demonstrates the implementation of a Retrieval-Augmented Generation (RAG) pipeline using Llama 3.2 via the Ollama API. We began by preparing and splitting PDF documents into manageable chunks, then ingested these into a vector database with FAISS and Hugging Face embeddings for efficient retrieval.

We integrated the retriever with a conversation chain driven by an LLM, using customized system prompts to generate concise responses based on relevant documents. Additionally, we implemented a method for managing conversation history to maintain context in multi-turn interactions, along with calculating similarity scores between generated answers and source documents using SentenceTransformers.

Overall, this notebook serves as a guide for creating a locally deployable RAG application, effectively combining Llama 3.2 for inference and FAISS for document retrieval.